In [1]:
import os
import json
import cv2
import hashlib
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Set directory //  Convert JSON → YOLO TXT 

In [2]:
CLASS_MAP = {
    "container": 0,
    "truck": 1
}

BASE_DIR = "datasets_container"

SPLITS = ["train", "validation"]

OUTPUT_DIR = "yolo_dataset"

for split in SPLITS:
    os.makedirs(f"{OUTPUT_DIR}/{split}/images", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/{split}/labels", exist_ok=True)

# Create convert function
def convert_json_to_yolo(json_path, img_path, label_output):

    with open(json_path) as f:
        data = json.load(f)

    img = cv2.imread(img_path)
    h, w, _ = img.shape

    yolo_lines = []

    for obj in data["labels"]:

        label = obj["label_class"]

        if label not in CLASS_MAP:
            continue

        cls_id = CLASS_MAP[label]

        cx = obj["centre"]["x"] / w
        cy = obj["centre"]["y"] / h

        bw = obj["size"]["x"] / w
        bh = obj["size"]["y"] / h

        yolo_lines.append(f"{cls_id} {cx} {cy} {bw} {bh}")

    with open(label_output, "w") as f:
        f.write("\n".join(yolo_lines))

# Batch convert
for split in SPLITS:

    input_dir = f"{BASE_DIR}/{split}"

    for file in tqdm(os.listdir(input_dir)):

        if not file.endswith(".json"):
            continue

        json_path = f"{input_dir}/{file}"

        with open(json_path) as f:
            meta = json.load(f)

        img_name = meta["image_filename"]

        img_path = f"{input_dir}/{img_name}"

        out_img = f"{OUTPUT_DIR}/{split}/images/{img_name}"
        out_lbl = f"{OUTPUT_DIR}/{split}/labels/{img_name.replace('.jpg','.txt')}"

        os.system(f"cp {img_path} {out_img}")

        convert_json_to_yolo(json_path, img_path, out_lbl)

100%|██████████| 54/54 [00:00<00:00, 304.10it/s]


# Handle NEGATIVE Images

In [3]:
NEG_DIR = "datasets_container/negative"

for file in os.listdir(NEG_DIR):
    if file.endswith(".jpg"):

        src = f"{NEG_DIR}/{file}"
        dst = f"{OUTPUT_DIR}/train/images/{file}"

        os.system(f"cp {src} {dst}")

# Create dataset.yaml

In [4]:
dataset_yaml = f"""
path: {os.getcwd()}/{OUTPUT_DIR}

train: train/images
val: validation/images

nc: 2

names:
  0: container
  1: truck
"""

with open("container_dataset.yaml", "w") as f:
    f.write(dataset_yaml)

In [5]:
import torch
print(f"Pytorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS available: {torch.backends.mps.is_available()}")

Pytorch version: 2.9.1
CUDA available: False
MPS available: True


In [6]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")   # nano = faster drone inference

results = model.train(
    data="container_dataset.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    device="mps",     # GPU
    patience=15,
    workers=4,
    project="runs_container",
    name="container_yolo"
)

New https://pypi.org/project/ultralytics/8.4.10 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.8 🚀 Python-3.11.9 torch-2.9.1 MPS (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=container_dataset.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=container_yolo, nbs=64, nms=False, opset=N

2026/02/03 19:37:03 INFO mlflow.tracking.fluent: Experiment with name 'runs_container' does not exist. Creating a new experiment.
2026/02/03 19:37:03 WARNING mlflow.spark: With Pyspark >= 3.2, PYSPARK_PIN_THREAD environment variable must be set to false for Spark datasource autologging to work.
2026/02/03 19:37:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.


MLflow: logging run_id(f16a085c4fd64071b63e12358314d239) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/miftahhadiyannoor/Documents/logistics-rag/notebooks/runs/detect/runs_container/container_yolo
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      1/100      6.24G      1.678      3.184      1.387        295        640: 100% ━━━━━━━━━━━━ 16/16 1.6s/it 25.4s1.5s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 4.7s/it 4.7s
                   all         27        403    0.00923      0.456     0.0449      0.023

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/100      8.32G      1.534      1.665      1.255        284        640: 1

In [7]:
import torch
print(f"Pytorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS available: {torch.backends.mps.is_available()}")

Pytorch version: 2.9.1
CUDA available: False
MPS available: True


# Export Best Model → models/container_yolov8.pt

In [10]:
# Export best model to models/container_yolov8.pt

import shutil

best_model = os.path.join("runs/detect/runs_container/container_yolo/weights/best.pt")
TARGET_PATH = os.path.join("models", "container_yolov8.pt")

# Copy best model to target path
shutil.copy(best_model, TARGET_PATH)
print(f"YOLOV8 best model has been exported to {TARGET_PATH} ✅")

YOLOV8 best model has been exported to models/container_yolov8.pt ✅


# Quick Validation Test

In [11]:
model = YOLO(TARGET_PATH)
img = "datasets_container/validation/3de22b94583ee0f7defd6c5a6ce439dc2610efad.jpg" # Example image path

# Perform inference
results = model.predict(source=img, conf=0.4, save=True)
results[0].plot()


image 1/1 /Users/miftahhadiyannoor/Documents/logistics-rag/notebooks/datasets_container/validation/3de22b94583ee0f7defd6c5a6ce439dc2610efad.jpg: 640x640 5 containers, 58.0ms
Speed: 6.7ms preprocess, 58.0ms inference, 16.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /Users/miftahhadiyannoor/Documents/logistics-rag/notebooks/runs/detect/predict


array([[[227, 121,  12],
        [231, 114,  10],
        [237, 117,  14],
        ...,
        [ 10,   7,  16],
        [ 10,   7,  16],
        [ 10,   7,  16]],

       [[255,  42,   4],
        [255,  42,   4],
        [255,  42,   4],
        ...,
        [ 10,   7,  16],
        [ 10,   7,  16],
        [ 10,   7,  16]],

       [[255,  42,   4],
        [255,  42,   4],
        [255,  42,   4],
        ...,
        [ 10,   7,  16],
        [ 10,   7,  16],
        [ 10,   7,  16]],

       ...,

       [[ 14,   2,  14],
        [ 14,   2,  14],
        [ 14,   3,  13],
        ...,
        [  7,   9,  20],
        [  8,   9,  23],
        [  9,  11,  22]],

       [[ 16,   2,  14],
        [ 16,   2,  14],
        [ 14,   3,  13],
        ...,
        [  0,   0,  28],
        [  0,   2,  33],
        [  1,   5,  34]],

       [[ 16,   2,  14],
        [ 16,   2,  14],
        [ 14,   3,  13],
        ...,
        [ 37,  42,  81],
        [ 34,  39,  78],
        [ 31,  36,  75]]